In [120]:
import requests
import pandas as pd
import os
import re
import math
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from collections import Counter
from natsort import natsorted

In [163]:
episodes = []
title=[]
types = []
popularity = []
document=[]
# positioning
folder = r"./tsv_anime/"
# iter over the file
for anime in tqdm(natsorted(os.listdir(folder))):
    df = pd.read_csv(folder+anime, sep = "\t")
    episodes.append(df["animeNumEpisode"][0])
    types.append(df["animeType"][0])
    popularity.append(df["animePopularity"][0])
    document.append(df["animeDescription"][0])
    title.append(df["animeTitle"][0])
input_episodes=12
input_types="TV"
popularity=245
print(df)

100%|█████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:03<00:00, 480.48it/s]

               animeTitle animeType  animeNumEpisode releaseDate     endDate  \
0  Konjiki no Gash Bell!!        TV              150  2003-04-06  2006-03-26   

   animeNumMembers  animeScore  animeUsers  animeRank  animePopularity  \
0           107912        7.54       46265       1499             1519   

                                    animeDescription  \
0  Takamine Kiyomaro, a depressed don't-care-abou...   

                                        animeRelated  \
0  ['Konjiki no Gash!!', 'Konjiki no Gash Bell!!:...   

                                     animeCharacters  \
0  ['Bell, Gash', 'Takamine, Kiyomaro', 'Folgore,...   

                                         animeVoices  \
0  ['Ootani, IkueJapanese', 'Sakurai, TakahiroJap...   

                                          animeStaff  
0  [[['Hamano, Takatoshi', 'Producer'], ['Kaizawa...  


In [122]:
df

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Konjiki no Gash Bell!!,TV,150,2003-04-06,2006-03-26,107912,7.54,46265,1499,1519,"Takamine Kiyomaro, a depressed don't-care-abou...","['Konjiki no Gash!!', 'Konjiki no Gash Bell!!:...","['Bell, Gash', 'Takamine, Kiyomaro', 'Folgore,...","['Ootani, IkueJapanese', 'Sakurai, TakahiroJap...","[[['Hamano, Takatoshi', 'Producer'], ['Kaizawa..."


In [171]:
#types

In [64]:
def type_cosine(input_type,next_type):
    WORD = re.compile(r"\w+")
    intersection = set(input_type.keys()) & set(next_type.keys())
    numerator = sum([input_type[x] * next_type[x] for x in intersection])
    sum1 = sum([input_type[x] ** 2 for x in list(input_type.keys())])
    sum2 = sum([input_type[x] ** 2 for x in list(next_type.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator  

In [65]:
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [66]:
def type_score(input_type,next_type):
    vector1 = text_to_vector(input_type)
    vector2 = text_to_vector(next_type)
    cosine = type_cosine(vector1, vector2)
    return(cosine)

In [67]:
text1="Gintoki, Shinpachi, and Kagura return as the fun-loving but broke members of the Yorozuya team! Living in an alternate-reality Edo, where swords are prohibited and alien overlords have conquered Japan, they try to thrive on doing whatever work they can get their hands on. However, Shinpachi and Kagura still haven't been paid... Does Gin-chan really spend all that cash playing pachinko?\n\n\nMeanwhile, when Gintoki drunkenly staggers home one night, an alien spaceship crashes nearby. A fatally injured crew member emerges from the ship and gives Gintoki a strange, clock-shaped device, warning him that it is incredibly powerful and must be safeguarded. Mistaking it for his alarm clock, Gintoki proceeds to smash the device the next morning and suddenly discovers that the world outside his apartment has come to a standstill. With Kagura and Shinpachi at his side, he sets off to get the device fixed; though, as usual, nothing is ever that simple for the Yorozuya team.\n\n\nFilled with tongue-in-cheek humor and moments of heartfelt emotion, Gintama's fourth season finds Gintoki and his friends facing both their most hilarious misadventures and most dangerous crises yet.\n\n\n[Written by MAL Rewrite]"
text2='The self-proclaimed mad scientist Rintarou Okabe rents out a room in a rickety old building in Akihabara, where he indulges himself in his hobby of inventing prospective "future gadgets" with fellow lab members: Mayuri Shiina, his air-headed childhood friend, and Hashida Itaru, a perverted hacker nicknamed "Daru." The three pass the time by tinkering with their most promising contraption yet, a machine dubbed the "Phone Microwave," which performs the strange function of morphing bananas into piles of green gel.\n\n\nThough miraculous in itself, the phenomenon doesn\'t provide anything concrete in Okabe\'s search for a scientific breakthrough; that is, until the lab members are spurred into action by a string of mysterious happenings before stumbling upon an unexpected success—the Phone Microwave can send emails to the past, altering the flow of history.\n\n\nAdapted from the critically acclaimed visual novel by 5pb. and Nitroplus, Steins;Gate takes Okabe through the depths of scientific theory and practicality. Forced across the diverging threads of past and present, Okabe must shoulder the burdens that come with holding the key to the realm of time.\n\n\n[Written by MAL Rewrite]'
type_score(text1,text2)

0.7199269454413546

In [145]:
def identify_eps(input_eps,next_eps):
    if(type(next_eps) == str):
        return 0
    score=0
    #0 is film
    #1 is short
    #2 is medium
    #3 is large

    if(input_eps==1):
        category=0
    elif(input_eps<25 and input_eps>=8):
        category=1
    elif(input_eps<65 and input_eps>=25):
        category=2
    else:
        category=3
        
    if(next_eps==1):
        category1=0
    elif(next_eps<25 and input_eps>=8):
        category1=1
    elif(next_eps<65 and input_eps>=25):
        category1=2
    else:
        category1=3
    return(category,category1)

In [144]:
def episode_score(input_eps,next_eps):
    if(type(next_eps)==str):
        return 0
    category=(identify_eps(input_eps,next_eps))
    #print(category)
    if(category[0]==category[1]):
        return 1
    elif(abs(category[0]-category[1])==1):
        return 0.66
    elif(abs(category[0]-category[1])>1):
        return 0.33
    else:
        return 0

In [140]:
print(episode_score(27,63))

1


In [72]:
#def episode_score(input_eps,next_eps):
    #min_eps=1
    #max_eps=1000
    #diff=abs(input_eps-next_eps)
    #return diff

In [174]:
def final_score(i,input_type,next_type,input_eps,next_eps):
    #print(i)
    score1=type_score(input_type,next_type)
    score2=episode_score(input_eps,next_eps)
    return(score1+score2)

In [128]:
len(episodes)

1500

In [173]:
#for i in range(len(episodes)):
    #print(episodes[i],type(episodes[i]))

In [143]:
type(episodes[70])

str

In [175]:
type_query="TV"
ep_query=13
scores=[]
for i in range(len(episodes)):
    score=final_score(i,type_query,types[i],ep_query,episodes[i])
    scores.append([score,i])
sorted_scores=sorted(scores, reverse=True)

for i in range(len(episodes)):
    #print(sorted_scores[i][1])
    print(sorted_scores[i],title[sorted_scores[i][1]])

[2.0, 1489] Fugou Keiji: Balance:Unlimited
[2.0, 1485] D-Frag!
[2.0, 1480] Majo no Tabitabi
[2.0, 1478] Kemono Friends
[2.0, 1476] Isshuukan Friends.
[2.0, 1472] Hataraku Saibou Black (TV)
[2.0, 1461] Yama no Susume: Second Season
[2.0, 1458] Tamayura: More Aggressive
[2.0, 1457] Sword Art Online: Alicization
[2.0, 1456] Shoukoku no Altair
[2.0, 1455] Shin Tennis no Ouji-sama
[2.0, 1452] Yondemasu yo, Azazel-san. Z
[2.0, 1450] Yama no Susume: Third Season
[2.0, 1447] Tonari no Seki-kun
[2.0, 1445] Tada-kun wa Koi wo Shinai
[2.0, 1441] Re:Creators
[2.0, 1433] Mitsudomoe
[2.0, 1432] Mashiro no Oto
[2.0, 1427] Itai no wa Iya nano de Bougyoryoku ni Kyokufuri Shitai to Omoimasu.
[2.0, 1416] Densetsu no Yuusha no Densetsu
[2.0, 1414] Chuunibyou demo Koi ga Shitai! Ren
[2.0, 1413] Black Jack 21
[2.0, 1412] Basilisk: Kouga Ninpou Chou
[2.0, 1411] Baka to Test to Shoukanjuu
[2.0, 1408] Love Live! Sunshine!! 2nd Season
[2.0, 1407] Love Live! Nijigasaki Gakuen School Idol Doukoukai
[2.0, 1406] Ju